# PREDICCION MAHER (BIVARIATE POISSON)

DESCRIPCION DEL MODELO

In [1]:
import pandas as pd
from scipy.stats import poisson
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore')
pd.options.display.max_columns = None #quitar limite de visualizacion de columnas
import matplotlib.pyplot as plt
from scipy.stats import poisson,skellam
from scipy.optimize import minimize
import math

### NOS BASAMOS EN LAS ULTIMAS 4 TEMPORADAS 17/18, 18/19, 19/20, 20/21 PARA TRATAR DE PREDECIR LA TEMPORADA 21/22 

In [2]:
resultados = pd.read_csv('resultados_93_23_la_liga.csv')
resultados_previos = resultados[((resultados.Temporada == '20/21') | (resultados.Temporada == '19/20') | (resultados.Temporada == '18/19') | (resultados.Temporada == '17/18'))]
resultados_a_predecir = pd.read_csv('resultados_a_predecir_21_22_i.csv')

In [3]:
resultados_previos.head(2)

,Temporada,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
450,20/21,La Liga,2021-05-23,Granada,Getafe,0,0,D
451,20/21,La Liga,2021-05-23,Sevilla,Alaves,1,0,H


In [4]:
resultados_a_predecir.head(2)

,Temporada,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,FTR_predict_i
0,21/22,La Liga,2022-05-22,Sevilla,Ath Bilbao,1,0,H,H
1,21/22,La Liga,2022-05-22,Barcelona,Villarreal,0,2,A,H


In [5]:
print(len(resultados_previos),'partidos disputados')

1520 partidos disputados


In [6]:
stats = pd.read_csv('stats_4temporadas_17_21_i.csv')
stats

,Equipo,VL,DL,EL,VV,DV,EV,FAL_i,FDL_i,FAV_i,FDV_i
0,La Coruna,4,9,6,2,12,5,-0.202176,0.556604,-0.245283,0.559383
1,Malaga,4,12,3,1,16,2,-0.492294,0.273585,-0.528302,0.233001
2,Las Palmas,4,13,2,1,13,5,-0.456029,0.792453,-0.575472,0.305530
3,Rayo Vallecano,5,8,6,3,14,2,-0.093382,0.509434,-0.245283,0.378060
4,Girona,11,18,9,12,16,10,-0.220308,0.179245,0.037736,0.124207
5,Mallorca,8,8,3,1,15,3,-0.093382,0.037736,-0.292453,0.559383
6,Leganes,21,19,17,10,33,14,-0.335147,-0.040881,-0.276730,0.015413
7,Espanol,21,21,15,10,28,19,-0.238441,0.084906,-0.245283,-0.020852
8,Villarreal,33,21,22,28,30,18,0.169538,0.132075,0.179245,-0.102448
9,Valencia,39,10,27,22,34,20,0.115141,-0.209906,0.049528,0.015413


In [7]:
import math
    
def likelihood(FAL, FDL, FAV, FDV, GL, GV):
    a = math.exp(FAL + FDV)
    b = math.exp(FAV + FDL)
    return -a+GL*math.log(a)-b+GV*math.log(b)

def sumatorio_likelihoods(params):
    temp = []
    for i, teamA in enumerate(stats.Equipo):
        for j, teamB in enumerate(stats.Equipo):
            if teamA != teamB and len(resultados_previos[((resultados_previos.HomeTeam == teamA) & (resultados_previos.AwayTeam == teamB))]) != 0:
                FAL = params[i]
                FDL = params[len(stats)+i]
                FAV = params[2*len(stats)+j]
                FDV = params[3*len(stats)+j]
                GL = resultados_previos[((resultados_previos.HomeTeam == teamA) & (resultados_previos.AwayTeam == teamB))].FTHG.iloc[0]
                GV = resultados_previos[((resultados_previos.HomeTeam == teamA) & (resultados_previos.AwayTeam == teamB))].FTAG.iloc[0]
                #print('GL:', GL, teamA, 'FAL:', FAL, 'FDL:', FDL, '\nGV:', GV, teamB, 'FAV:',  FAV, 'FDV:', FDV,'\n\n')
                temp.append(likelihood(FAL, FDL, FAV, FDV, GL, GV))
    return -sum(temp)

In [8]:
params = [0.5 for i in range(len(stats))] + [-0.5 for i in range(len(stats))] + [0.5 for i in range(len(stats))] + [-0.5 for i in range(len(stats))]
len(params)

112

In [9]:
from scipy.optimize import minimize 
from datetime import datetime

start = datetime.now()
print(start)
resultados_minimize = minimize(sumatorio_likelihoods, params, method='SLSQP')

stop = datetime.now()
print(stop)
print(f'Time taken to run: {stop - start}')

2022-10-06 18:09:36.535515
2022-10-06 19:48:49.215884
Time taken to run: 1:39:12.680369


In [10]:
#stats = pd.read_csv('resultados_minimize_4temporadas_17_21.csv')

In [11]:
stats.insert(8, 'FAL_BP', resultados_minimize.x[:len(stats)])
stats.insert(10, 'FDL_BP', resultados_minimize.x[len(stats):len(stats)*2])
stats.insert(12, 'FAV_BP', resultados_minimize.x[len(stats)*2:len(stats)*3])
stats.insert(14, 'FDV_BP', resultados_minimize.x[len(stats)*3:])

In [12]:
stats

,Equipo,VL,DL,EL,VV,DV,EV,FAL_i,FAL_BP,FDL_i,FDL_BP,FAV_i,FAV_BP,FDV_i,FDV_BP
0,La Coruna,4,9,6,2,12,5,-0.202176,0.582959,0.556604,-0.059772,-0.245283,0.319366,0.559383,0.105871
1,Malaga,4,12,3,1,16,2,-0.492294,0.111159,0.273585,-0.275474,-0.528302,-0.166264,0.233001,-0.146253
2,Las Palmas,4,13,2,1,13,5,-0.456029,0.184323,0.792453,0.064467,-0.575472,-0.245288,0.305530,-0.087078
3,Rayo Vallecano,5,8,6,3,14,2,-0.093382,0.748688,0.509434,-0.126989,-0.245283,0.359977,0.378060,-0.034611
4,Girona,11,18,9,12,16,10,-0.220308,0.545660,0.179245,-0.333111,0.037736,0.532396,0.124207,-0.484667
5,Mallorca,8,8,3,1,15,3,-0.093382,0.729155,0.037736,-0.520686,-0.292453,0.300086,0.559383,0.070508
6,Leganes,21,19,17,10,33,14,-0.335147,0.167875,-0.040881,-0.467636,-0.276730,0.220313,0.015413,-0.549402
7,Espanol,21,21,15,10,28,19,-0.238441,0.459169,0.084906,-0.134450,-0.245283,0.284707,-0.020852,-0.442848
8,Villarreal,33,21,22,28,30,18,0.169538,0.778934,0.132075,-0.439703,0.179245,0.947849,-0.102448,-0.585529
9,Valencia,39,10,27,22,34,20,0.115141,0.948506,-0.209906,-0.604917,0.049528,0.431422,0.015413,-0.160728


In [13]:
stats.to_csv('stats_4temporadas_17_21_i_BP.csv', index = False)

In [14]:
def simular_partido(stats, local, visitante, modelo='i'):
    
    FAL, FDL, FAV, FDV = 'FAL_'+ modelo, 'FDL_'+ modelo, 'FAV_'+ modelo, 'FDV_'+ modelo
    
    FAL = stats[stats.Equipo == local][FAL].iloc[0]
    FDL = stats[stats.Equipo == local][FDL].iloc[0]
    FAV = stats[stats.Equipo == visitante][FAV].iloc[0]
    FDV = stats[stats.Equipo == visitante][FDV].iloc[0]
        
    objetivo_esperado_local = math.exp(FAL + FDV)
    objetivo_esperado_visitante = math.exp(FAV + FDL)
    '''print('EN EL MODELO', modelo, ':\n--------------------------')
    print('Se espera que el', local, 'marque', round(objetivo_esperado_local,3), 'goles contra el', visitante)
    print('Se espera que el', visitante, 'marque', round(objetivo_esperado_visitante,3), 'goles contra el', local,'\n')'''
    team_local_poisson = [poisson.pmf(i, objetivo_esperado_local) for i in range(10)]
    team_visitante_poisson = [poisson.pmf(i, objetivo_esperado_visitante) for i in range(10)]
    partido = np.outer(team_local_poisson, team_visitante_poisson)
    tabla = pd.DataFrame(partido)
    victoria_local = np.sum(np.tril(partido, -1))
    empate = np.sum(np.diag(partido))
    victoria_visit = np.sum(np.triu(partido, 1))

    '''print('Prob implicita victoria local:', round(victoria_local*100,2), '%')
    print('Prob implicita Empate:', round(empate*100, 2), '%')
    print('Prob implicita victoria visitante:', round(victoria_visit*100,2), '%\n')'''
    
    res = max(victoria_local, empate, victoria_visit)
    if res ==  victoria_local: FTR = 'H'
    if res == empate: FTR = 'D'
    if res == victoria_visit: FTR = 'A'
    
    return FTR, tabla

local = 'Sevilla'
visitante = 'Ath Bilbao'

#FTR, simulacion_resultados_inical = simular_partido(stats, local, visitante, modelo='i')
FTR, simulacion_resultados_final = simular_partido(stats, local, visitante, modelo='BP')

In [15]:
#Sevilla vs Ath Bilbao
simulacion_resultados_final

,0,1,2,3,4,5,6,7,8,9
0,0.154060,0.065882,0.014087,2.007985e-03,2.146713e-04,1.836020e-05,1.308578e-06,7.994204e-08,4.273255e-09,2.030438e-10
1,0.222275,0.095052,0.020324,2.897070e-03,3.097224e-04,2.648964e-05,1.887984e-06,1.153384e-07,6.165346e-09,2.929466e-10
2,0.160346,0.068570,0.014661,2.089911e-03,2.234299e-04,1.910930e-05,1.361969e-06,8.320369e-08,4.447604e-09,2.113281e-10
3,0.077114,0.032977,0.007051,1.005091e-03,1.074530e-04,9.190143e-06,6.550049e-07,4.001474e-08,2.138965e-09,1.016330e-10
4,0.027815,0.011895,0.002543,3.625299e-04,3.875765e-05,3.314828e-06,2.362562e-07,1.443307e-08,7.715113e-10,3.665838e-11
5,0.008026,0.003432,0.000734,1.046098e-04,1.118372e-05,9.565103e-07,6.817292e-08,4.164735e-09,2.226235e-10,1.057796e-11
6,0.001930,0.000825,0.000176,2.515475e-05,2.689265e-06,2.300049e-07,1.639304e-08,1.001463e-09,5.353261e-11,2.543604e-12
7,0.000398,0.000170,0.000036,5.184665e-06,5.542864e-07,4.740648e-08,3.378781e-09,2.064123e-10,1.103365e-11,5.242641e-13
8,0.000072,0.000031,0.000007,9.350382e-07,9.996383e-08,8.549613e-09,6.093527e-10,3.722581e-11,1.989884e-12,9.454940e-14
9,0.000012,0.000005,0.000001,1.498944e-07,1.602504e-08,1.370574e-09,9.768433e-11,5.967609e-12,3.189950e-13,1.515706e-14


In [16]:
def resultados_mas_probables(tabla):
    aux = tabla.copy()
    tops =[]
    df = pd.DataFrame(columns=['local', 'visitante', 'prob (%)'])
    prob = 0
    for _ in range(10):
        top=[0,0,prob]
        for col in range(aux.shape[1]):
            for fil in range(aux.shape[0]):
                prob = aux.iloc[fil,col]
                if prob > top[-1]: top = [fil,col,prob]
        aux.iloc[top[0], top[1]]=0
        tops.append(top)
        df = df.append({'local': top[0], 'visitante':top[1], 'prob (%)':round(top[-1]*100,2)}, ignore_index=True)
        df.local = df.local.astype('int64')
        df.visitante = df.visitante.astype('int64')
        
    return df, (df.local.iloc[0], df.visitante.iloc[0])

top10_predict, resultado_predict = resultados_mas_probables(simulacion_resultados_final)

In [17]:
top10_predict

,local,visitante,prob (%)
0,1,0,22.23
1,2,0,16.03
2,0,0,15.41
3,1,1,9.51
4,3,0,7.71
5,2,1,6.86
6,0,1,6.59
7,3,1,3.3
8,4,0,2.78
9,1,2,2.03


In [18]:
resultados_a_predecir

,Temporada,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,FTR_predict_i
0,21/22,La Liga,2022-05-22,Sevilla,Ath Bilbao,1,0,H,H
1,21/22,La Liga,2022-05-22,Barcelona,Villarreal,0,2,A,H
2,21/22,La Liga,2022-05-22,Osasuna,Mallorca,0,2,A,H
3,21/22,La Liga,2022-05-22,Granada,Espanol,0,0,D,H
4,21/22,La Liga,2022-05-22,Alaves,Cadiz,0,1,A,A
...,...,...,...,...,...,...,...,...,...
375,21/22,La Liga,2021-08-14,Osasuna,Espanol,0,0,D,A
376,21/22,La Liga,2021-08-14,Alaves,Real Madrid,1,4,A,A
377,21/22,La Liga,2021-08-14,Mallorca,Betis,1,1,D,A
378,21/22,La Liga,2021-08-14,Cadiz,Levante,1,1,D,A


In [19]:
def predict(resultados, stats, modelo):
    resultados_predict = []
    for i in range(len(resultados)):
        local = resultados.HomeTeam.iloc[i]
        visitante = resultados.AwayTeam.iloc[i]
        FTR = simular_partido(stats, local, visitante, modelo)[0]
        #print('\n')
        resultados_predict.append(FTR)
    resultados_a_predecir['FTR_predict_'+modelo] = resultados_predict
    return resultados_a_predecir
predict(resultados_a_predecir, stats, 'BP')

,Temporada,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,FTR_predict_i,FTR_predict_BP
0,21/22,La Liga,2022-05-22,Sevilla,Ath Bilbao,1,0,H,H,H
1,21/22,La Liga,2022-05-22,Barcelona,Villarreal,0,2,A,H,H
2,21/22,La Liga,2022-05-22,Osasuna,Mallorca,0,2,A,H,H
3,21/22,La Liga,2022-05-22,Granada,Espanol,0,0,D,H,H
4,21/22,La Liga,2022-05-22,Alaves,Cadiz,0,1,A,A,H
...,...,...,...,...,...,...,...,...,...,...
375,21/22,La Liga,2021-08-14,Osasuna,Espanol,0,0,D,A,H
376,21/22,La Liga,2021-08-14,Alaves,Real Madrid,1,4,A,A,A
377,21/22,La Liga,2021-08-14,Mallorca,Betis,1,1,D,A,H
378,21/22,La Liga,2021-08-14,Cadiz,Levante,1,1,D,A,A


In [20]:
predict(resultados_a_predecir, stats, 'BP').to_csv('resultados_a_predecir_21_22_i_BP.csv', index=False)

In [21]:
def fiabilidad(resultados, modelo):
    aciertos = 0
    home = 0
    draw = 0
    away = 0
    for i in range(len(resultados)):
        if resultados.FTR.iloc[i] == resultados['FTR_predict_'+modelo].iloc[i]:
            if resultados.FTR.iloc[i] == 'H': home+=1 
            if resultados.FTR.iloc[i] == 'D': draw+=1
            if resultados.FTR.iloc[i] == 'A': away+=1
            aciertos += 1

    print('----------------------------------------------------')
    print('FIABILIDAD MODELO', modelo,':')
    print('----------------------------------------------------')
    print(aciertos, '/', len(resultados), 'resultados acertados \t\t--->',round(aciertos/len(resultados)*100,2), '%')
    print('----------------------------------------------------')
    print(home + away, '/', len(resultados[resultados.FTR != 'D']), 'victorias/derrotas acertadas \t--->',round((home+away)/len(resultados[resultados.FTR != 'D'])*100,2), '%')
    print(draw, '/', len(resultados[resultados.FTR == 'D']), 'empates acertados \t\t--->',round(draw/len(resultados[resultados.FTR == 'D'])*100,2), '%')
fiabilidad(predict(resultados_a_predecir, stats, 'BP'), 'BP')

----------------------------------------------------
FIABILIDAD MODELO BP :
----------------------------------------------------
192 / 380 resultados acertados 		---> 50.53 %
----------------------------------------------------
191 / 269 victorias/derrotas acertadas 	---> 71.0 %
1 / 111 empates acertados 		---> 0.9 %
